# Classification

In [3]:
set.seed(37)

In [26]:
suppressMessages({
    library('dplyr')
})

getData <- function(N=1000) {
    x1 <- rnorm(N, mean=0, sd=1)
    x2 <- rnorm(N, mean=0, sd=1)
    y <- 1 + 2.0 * x1 + 3.0 * x2 + rnorm(N, mean=0, sd=1)
    y <- 1.0 / (1.0 + exp(-y))
    y <- rbinom(n=N, size=1, prob=y)
    
    df <- data.frame(x1=x1, x2=x2, y=y)
    df <- df %>% 
            mutate(y=ifelse(y == 0, 'neg', 'pos')) %>%
            mutate_if(is.character, as.factor)
    return(df)
}

T <- getData()

In [27]:
summary(T)

       x1                 x2              y      
 Min.   :-3.61605   Min.   :-3.158035   neg:399  
 1st Qu.:-0.66954   1st Qu.:-0.709333   pos:601  
 Median : 0.01997   Median :-0.008304            
 Mean   : 0.01737   Mean   : 0.008245            
 3rd Qu.: 0.66307   3rd Qu.: 0.740304            
 Max.   : 2.93000   Max.   : 2.839198            

In [28]:
library('caret')

In [29]:
m <- train(y ~ ., method='rf', data=T)

note: only 1 unique complexity parameters in default grid. Truncating the grid to 1 .



ERROR: Error: package e1071 is required


In [ ]:
e1071 